# Storm surge level

Notebook environment to migrate netcdf files to CF compliant zarr

In [1]:
# Optional; code formatter, installed as jupyter lab extension
#%load_ext lab_black
# Optional; code formatter, installed as jupyter notebook extension
%load_ext nb_black

<IPython.core.display.Javascript object>

### Configure OS independent paths

In [2]:
# Import standard packages
import os
import pathlib
import sys
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr

# Make root directories importable by appending root to path
cwd = pathlib.Path().resolve()
sys.path.append(os.path.dirname(cwd))

# Get root paths
home = pathlib.Path().home()
root = home.root

# Import custom functionality
from etl import p_drive
from etl.CF_compliancy_checker import check_compliancy, save_compliancy

# Define (local and) remote drives
coclico_data_dir = p_drive.joinpath("11205479-coclico", "data")

# Workaround to the Windows OS (10) udunits error after installation of cfchecker: https://github.com/SciTools/iris/issues/404
os.environ["UDUNITS2_XML_PATH"] = str(
    home.joinpath(  # change to the udunits2.xml file dir in your Python installation
        r"Anaconda3\pkgs\udunits2-2.2.28-h892ecd3_0\Library\share\udunits\udunits2.xml"
    )
)

C:\Users\kras\Anaconda3\envs\bathymetry_env\lib\site-packages\xarray\backends\cfgrib_.py:27: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(


<IPython.core.display.Javascript object>

In [3]:
# Project paths & files (manual input)
dataset_dir = coclico_data_dir.joinpath("01_storm_surge_jrc")
dataset_historical_path = dataset_dir.joinpath("CoastAlRisk_Europe_EESSL_Historical.nc")
dataset_rcp45_path = dataset_dir.joinpath("CoastAlRisk_Europe_EESSL_RCP45.nc")
dataset_rcp85_path = dataset_dir.joinpath("CoastAlRisk_Europe_EESSL_RCP85.nc")
dataset_out_file = "CoastAlRisk_Europe_EESSL"
CF_dir = coclico_data_dir.joinpath(r"CF")  # directory to save output CF check files

<IPython.core.display.Javascript object>

### Check CF compliancy original NetCDF files

In [40]:
# open datasets
dataset_historical = xr.open_dataset(dataset_historical_path)
dataset_45rcp = xr.open_dataset(dataset_rcp45_path)
dataset_85rcp = xr.open_dataset(dataset_rcp85_path)

# check original dataset
dataset_historical

<xarray.Dataset>
Dimensions:    (row: 2242, col: 8)
Dimensions without coordinates: row, col
Data variables:
    longitude  (row) float64 ...
    latitude   (row) float64 ...
    ssl        (row, col) float64 ...
    RP         (col) float32 ...
Attributes:
    title:            European extreme storm surge level
    Institution:      Joint European Research Center, Institute of Environmen...
    Project Name:     Prototype of a first Global Integrated Coastal Impact-b...
    Project Acronym:  CoastAlRisk
    reference:        Vousdoukas MI, Voukouvalas E, Annunziato A, Giardino A,...
    email:            michail.vousdoukas@ec.europa.eu
    version:          1.0
    terms_for_use:    European Union, 1995-2015.\nReuse is authorised, provid...
    disclaimer:       Unless the following would not be permitted or valid un...

<IPython.core.display.Javascript object>

In [41]:
%%capture cap --no-stderr
# check original CF compliancy

check_compliancy(testfile=dataset_historical_path, working_dir=CF_dir)

<IPython.core.display.Javascript object>

In [42]:
# save original CF compliancy
save_compliancy(cap, testfile=dataset_historical_path, working_dir=CF_dir)

{'P:\\11205479-coclico\\data\\01_storm_surge_jrc\\CoastAlRisk_Europe_EESSL_Historical.nc': {'warnings': '3', 'errors': '2'}}


<IPython.core.display.Javascript object>

In [43]:
%%capture cap --no-stderr
# check original CF compliancy

check_compliancy(testfile=dataset_rcp45_path, working_dir=CF_dir)

<IPython.core.display.Javascript object>

In [44]:
# save original CF compliancy
save_compliancy(cap, testfile=dataset_rcp45_path, working_dir=CF_dir)

{'P:\\11205479-coclico\\data\\01_storm_surge_jrc\\CoastAlRisk_Europe_EESSL_RCP45.nc': {'warnings': '3', 'errors': '2'}}


<IPython.core.display.Javascript object>

In [45]:
%%capture cap --no-stderr
# check original CF compliancy

check_compliancy(testfile=dataset_rcp85_path, working_dir=CF_dir)

<IPython.core.display.Javascript object>

In [46]:
# save original CF compliancy
save_compliancy(cap, testfile=dataset_rcp85_path, working_dir=CF_dir)

{'P:\\11205479-coclico\\data\\01_storm_surge_jrc\\CoastAlRisk_Europe_EESSL_RCP85.nc': {'warnings': '3', 'errors': '2'}}


<IPython.core.display.Javascript object>

### Make CF compliant alterations to the NetCDF files (dataset dependent)

In [47]:
# NetCDF attribute alterations

# rename global attribute names
dataset_historical.attrs["Project_Name"] = dataset_historical.attrs.pop("Project Name")
dataset_historical.attrs["Project_Acronym"] = dataset_historical.attrs.pop(
    "Project Acronym"
)
dataset_45rcp.attrs["Project_Name"] = dataset_45rcp.attrs.pop("Project Name")
dataset_45rcp.attrs["Project_Acronym"] = dataset_45rcp.attrs.pop("Project Acronym")
dataset_85rcp.attrs["Project_Name"] = dataset_85rcp.attrs.pop("Project Name")
dataset_85rcp.attrs["Project_Acronym"] = dataset_85rcp.attrs.pop("Project Acronym")

# add global attributes
dataset_historical.attrs["Conventions"] = "CF-1.8"
dataset_45rcp.attrs["Conventions"] = "CF-1.8"
dataset_45rcp.attrs["Conventions"] = "CF-1.8"
dataset_historical.attrs["Starting_date"] = "01-Dec-1969"
dataset_historical.attrs["End_date"] = "30-Nov-2004 21:00:00"
dataset_45rcp.attrs["Starting_date"] = "01-Dec-2009"
dataset_45rcp.attrs["End_date"] = "30-Nov-2099 21:00:00"
dataset_85rcp.attrs["Starting_date"] = "01-Dec-2009"
dataset_85rcp.attrs["End_date"] = "30-Nov-2099 21:00:00"

# remove certain variable attributes
del dataset_historical["RP"].attrs["Starting date"]
del dataset_historical["RP"].attrs["End date"]
del dataset_45rcp["RP"].attrs["Starting date"]
del dataset_45rcp["RP"].attrs["End date"]
del dataset_85rcp["RP"].attrs["Starting date"]
del dataset_85rcp["RP"].attrs["End date"]

<IPython.core.display.Javascript object>

In [48]:
# NetCDF variable and dimension alterations

# rename or swap dimension names, the latter in case the name already exists as coordinate
dataset_historical = dataset_historical.rename_dims({"row": "stations"})
dataset_45rcp = dataset_45rcp.rename_dims({"row": "stations"})
dataset_85rcp = dataset_85rcp.rename_dims({"row": "stations"})
dataset_historical = dataset_historical.swap_dims({"col": "rp"})
dataset_45rcp = dataset_45rcp.swap_dims({"col": "rp"})
dataset_85rcp = dataset_85rcp.swap_dims({"col": "rp"})

# rename variables, if necessary
dataset_historical = dataset_historical.rename_vars(
    {"RP": "rp", "longitude": "lon", "latitude": "lat"}
)
dataset_45rcp = dataset_45rcp.rename_vars(
    {"RP": "rp", "longitude": "lon", "latitude": "lat"}
)
dataset_85rcp = dataset_85rcp.rename_vars(
    {"RP": "rp", "longitude": "lon", "latitude": "lat"}
)

# set some data variables to coordinates to avoid duplication of dimensions in later stage
dataset_historical = dataset_historical.set_coords(["lon", "lat", "rp"])
dataset_45rcp = dataset_45rcp.set_coords(["lon", "lat", "rp"])
dataset_85rcp = dataset_85rcp.set_coords(["lon", "lat", "rp"])

<IPython.core.display.Javascript object>

In [57]:
# concat datasets along new dimension with index values and name derived from pandas index object, if necessary
dataset = xr.concat(
    [dataset_historical, dataset_45rcp, dataset_85rcp], dim="nscenarios",
)
dataset = dataset.assign_coords(
    scenarios=("nscenarios", np.array(["historical", "rcp45", "rcp85"], dtype="S"))
)

# dataset = xr.concat(
#     [dataset_historical, dataset_45rcp, dataset_85rcp],
#     pd.Index(["historical", "rcp45", "rcp85"], name="scenarios"),
# )

# dataset["scenarios"].values.astype("U") # retrieve scenarios as string

<IPython.core.display.Javascript object>

In [90]:
# re-order shape of the data variables
dataset_historical = dataset_historical.transpose("stations", "rp")
dataset_45rcp = dataset_45rcp.transpose("stations", "rp")
dataset_85rcp = dataset_85rcp.transpose("stations", "rp")
dataset = dataset.transpose("nscenarios", "stations", "rp")

# add or change certain variable / coordinate attributes
dataset_attributes = {
    "scenarios": {"long_name": "climate scenarios"}
}  # specify custom (CF convention) attributes

# add / overwrite attributes
for k, v in dataset_attributes.items():
    try:
        dataset[k].attrs = dataset_attributes[k]
    except:
        continue

<IPython.core.display.Javascript object>

In [84]:
# check the xarray dataset, best practice is to have as many as possible bold dimensions (dimension == coordinate name).
# in this way, the Front-End can access the variable directly without having to index the variable first

dataset
# dataset["scenarios"]

<xarray.Dataset>
Dimensions:    (stations: 2242, nscenarios: 3, rp: 8)
Coordinates:
    lon        (stations) float64 -0.1 -0.1 -0.1 -0.1 -0.3 ... 9.9 9.9 9.9 9.9
    lat        (stations) float64 36.1 39.3 49.7 54.3 ... 57.7 58.7 64.5 64.7
  * rp         (rp) float32 5.0 10.0 20.0 50.0 100.0 200.0 500.0 1e+03
    scenarios  (nscenarios) |S10 b'historical' b'rcp45' b'rcp85'
Dimensions without coordinates: stations, nscenarios
Data variables:
    ssl        (nscenarios, stations, rp) float64 1.024 1.051 ... 2.712 2.805
Attributes:
    title:            European extreme storm surge level
    Institution:      Joint European Research Center, Institute of Environmen...
    reference:        Vousdoukas MI, Voukouvalas E, Annunziato A, Giardino A,...
    email:            michail.vousdoukas@ec.europa.eu
    version:          1.0
    terms_for_use:    European Union, 1995-2015.\nReuse is authorised, provid...
    disclaimer:       Unless the following would not be permitted or valid un...
    Project_Name:     Prototype of a first Global Integrated Coastal Impact-b...
    Project_Acronym:  CoastAlRisk
    Conventions:      CF-1.8
    Starting_date:    01-Dec-1969
    End_date:         30-Nov-2004 21:00:00

<IPython.core.display.Javascript object>

In [67]:
# save new .nc files
dataset_historical.to_netcdf(path=str(dataset_historical_path).replace(".nc", "_CF.nc"))
dataset_45rcp.to_netcdf(path=str(dataset_rcp45_path).replace(".nc", "_CF.nc"))
dataset_45rcp.to_netcdf(path=str(dataset_rcp85_path).replace(".nc", "_CF.nc"))
dataset.to_netcdf(path=dataset_dir.joinpath(dataset_out_file + "_CF.nc"))

<IPython.core.display.Javascript object>

### Check CF compliancy altered NetCDF files

In [68]:
%%capture cap --no-stderr
# check altered CF compliancy

check_compliancy(testfile=str(dataset_historical_path).replace(".nc", "_CF.nc"), working_dir=CF_dir)

<IPython.core.display.Javascript object>

In [69]:
# save altered CF compliancy
save_compliancy(
    cap,
    testfile=str(dataset_historical_path).replace(".nc", "_CF.nc"),
    working_dir=CF_dir,
)

{'P:\\11205479-coclico\\data\\01_storm_surge_jrc\\CoastAlRisk_Europe_EESSL_Historical_CF.nc': {'warnings': '0', 'errors': '0'}}


<IPython.core.display.Javascript object>

In [70]:
%%capture cap --no-stderr
# check altered CF compliancy

check_compliancy(testfile=str(dataset_rcp45_path).replace(".nc", "_CF.nc"), working_dir=CF_dir)

<IPython.core.display.Javascript object>

In [71]:
# save altered CF compliancy
save_compliancy(
    cap, testfile=str(dataset_rcp45_path).replace(".nc", "_CF.nc"), working_dir=CF_dir,
)

{'P:\\11205479-coclico\\data\\01_storm_surge_jrc\\CoastAlRisk_Europe_EESSL_RCP45_CF.nc': {'warnings': '0', 'errors': '0'}}


<IPython.core.display.Javascript object>

In [72]:
%%capture cap --no-stderr
# check altered CF compliancy

check_compliancy(testfile=str(dataset_rcp85_path).replace(".nc", "_CF.nc"), working_dir=CF_dir)

<IPython.core.display.Javascript object>

In [73]:
# save altered CF compliancy
save_compliancy(
    cap, testfile=str(dataset_rcp85_path).replace(".nc", "_CF.nc"), working_dir=CF_dir,
)

{'P:\\11205479-coclico\\data\\01_storm_surge_jrc\\CoastAlRisk_Europe_EESSL_RCP85_CF.nc': {'warnings': '0', 'errors': '0'}}


<IPython.core.display.Javascript object>

In [74]:
%%capture cap --no-stderr
# check altered CF compliancy

check_compliancy(testfile=dataset_dir.joinpath(dataset_out_file + "_CF.nc"), working_dir=CF_dir)

<IPython.core.display.Javascript object>

In [75]:
# save altered CF compliancy
save_compliancy(
    cap, testfile=dataset_dir.joinpath(dataset_out_file + "_CF.nc"), working_dir=CF_dir,
)

{'P:\\11205479-coclico\\data\\01_storm_surge_jrc\\CoastAlRisk_Europe_EESSL_CF.nc': {'warnings': '0', 'errors': '0'}}


<IPython.core.display.Javascript object>

### write data to Zarr files

In [76]:
# export to zarr in write mode (to overwrite if exists)
dataset.to_zarr(dataset_dir.joinpath("%s.zarr" % dataset_out_file), mode="w")

<IPython.core.display.Javascript object>